In [1]:
#Please  construct  a  new  dataset  by  either  adding  two  independent  variables  or  removing  
#two independent  variables  from  finalsample.dta  dataset.  If  you  choose  to  add  two  independent variables,
#you could add any two independent variables that you think help explain stock returns. If  you  choose  to 
#remove  two  independent  variables,  you  could  remove  any  two  independent variables that already exist 
#in the finalsample.dta dataset.

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as dates
from pandas_datareader import data
import statsmodels.api as sm
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import ElasticNetCV
plt.rcParams['figure.figsize'] = [20, 15]

In [2]:
#atq=total assets
#ceqq=common/ordinary equity
#cheq=cash and short term investments
#dlttq=total long term debt
#epspiq=eps including extraordinary items
#saleq=sales/turnover
#dvpspq=dividends per share
#
regdata=pd.read_stata('/Users/jimmyaspras/Downloads/finalsample.dta')
regdata.columns

Index(['gvkey', 'datadate', 'sic_2', 'lagdate', 'lagRet2', 'lagVOL2',
       'lagPrice2', 'lagMV2', 'lagShareturnover2', 'lagRet2_sic', 'lagRet12',
       'lagVOL12', 'lagShareturnover12', 'lagRet12_std', 'lagRet12_min',
       'lagRet12_max', 'lagRet12_sic', 'lagdatadate', 'atq', 'ceqq', 'cheq',
       'dlttq', 'epspiq', 'saleq', 'dvpspq', 'sp500_ret_d', 'nasdaq_ret_d',
       'r2000_ret_d', 'dollar_ret_d', 'VIX', 'yield_3m', 'yield_10y',
       'gdp_growth', 'Bull_ave', 'Bull_Bear', 'ret', 'debt', 'cash', 'sale',
       'BM', 'PE', 'div_p', 'loglagPrice2', 'loglagVOL12', 'loglagMV2',
       'logatq', 'loglagVOL2'],
      dtype='object')

In [3]:
regdata.dtypes

gvkey                        float64
datadate              datetime64[ns]
sic_2                        float64
lagdate               datetime64[ns]
lagRet2                      float64
lagVOL2                      float64
lagPrice2                    float64
lagMV2                       float64
lagShareturnover2            float64
lagRet2_sic                  float64
lagRet12                     float64
lagVOL12                     float64
lagShareturnover12           float64
lagRet12_std                 float64
lagRet12_min                 float64
lagRet12_max                 float64
lagRet12_sic                 float64
lagdatadate           datetime64[ns]
atq                          float64
ceqq                         float64
cheq                         float64
dlttq                        float64
epspiq                       float64
saleq                        float64
dvpspq                       float64
sp500_ret_d                  float64
nasdaq_ret_d                 float64
r

In [4]:
regdata.sort_values(by=['datadate'], inplace=True)
regdata1=regdata[regdata['lagPrice2']>=5]#remove penny stocks
regdata1['Year']=regdata1['datadate'].dt.year
regdata1['Month']=regdata1['datadate'].dt.month
#set gvkey and datadate as the index
regdata1=regdata1.set_index(['gvkey','datadate'])
regdata1.head()

<ipython-input-4-4039d9e85c2c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  regdata1['Year']=regdata1['datadate'].dt.year
<ipython-input-4-4039d9e85c2c>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  regdata1['Month']=regdata1['datadate'].dt.month


,,sic_2,lagdate,lagRet2,lagVOL2,lagPrice2,lagMV2,lagShareturnover2,lagRet2_sic,lagRet12,lagVOL12,...,BM,PE,div_p,loglagPrice2,loglagVOL12,loglagMV2,logatq,loglagVOL2,Year,Month
gvkey,datadate,,,,,,,,,,,,,,,,,,,,,
1004.0,2001-04-30,50.0,2001-02-28,-0.104674,1129800.0,13.600,366.27520,0.041950,-0.048423,0.052613,1481250.0,...,0.931715,85.000000,0.006250,2.610070,14.208397,5.903385,6.650203,13.937551,2001,4
30915.0,2001-04-30,33.0,2001-02-28,-0.050095,63300.0,10.050,91.80675,0.006929,0.004113,-0.018221,239500.0,...,1.624488,37.222222,0.000000,2.307573,12.386309,4.519686,5.489338,11.055641,2001,4
31170.0,2001-04-30,33.0,2001-02-28,-0.079365,296200.0,7.250,44.02925,0.048773,0.004113,-0.005784,345900.0,...,1.076057,34.523810,0.000000,1.981001,12.753905,3.784854,4.315018,12.598790,2001,4
31600.0,2001-04-30,33.0,2001-02-28,-0.035294,1058400.0,5.125,84.70600,0.064037,0.004113,0.078186,1158450.0,...,3.812655,36.607143,0.009756,1.634131,13.962593,4.439186,6.527946,13.872269,2001,4
61635.0,2001-04-30,33.0,2001-02-28,0.044444,201200.0,11.750,76.35150,0.030963,0.004113,0.318682,341450.0,...,1.379685,28.658537,0.000000,2.463853,12.740957,4.335348,5.667520,12.212055,2001,4


In [5]:
#Split  your  new  dataset  into  training  and  testing  samples.  Testing  sample  should  include  data 
#with year>=2016. 
#
#Drop dvpspq and atq from the train/test data
#
train=regdata1[regdata1['Year']<2016]
X_train=train[['sic_2', 'lagRet2', 'lagVOL2',
       'lagPrice2', 'lagMV2', 'lagShareturnover2', 'lagRet2_sic', 'lagRet12',
       'lagVOL12', 'lagShareturnover12', 'lagRet12_std', 'lagRet12_min',
       'lagRet12_max', 'lagRet12_sic', 'ceqq', 'cheq',
       'dlttq', 'epspiq', 'saleq', 'sp500_ret_d', 'nasdaq_ret_d',
       'r2000_ret_d', 'dollar_ret_d', 'VIX', 'yield_3m', 'yield_10y',
       'gdp_growth', 'Bull_ave', 'Bull_Bear', 'ret', 'debt', 'cash', 'sale',
       'BM', 'PE', 'div_p', 'loglagPrice2', 'loglagVOL12', 'loglagMV2',
       'logatq', 'loglagVOL2']]

In [6]:
#Set return as the dependent variable
Y_train=train[['ret']]

In [7]:
test=regdata1[regdata1['Year']>=2016]
X_test=test[['sic_2', 'lagRet2', 'lagVOL2',
       'lagPrice2', 'lagMV2', 'lagShareturnover2', 'lagRet2_sic', 'lagRet12',
       'lagVOL12', 'lagShareturnover12', 'lagRet12_std', 'lagRet12_min',
       'lagRet12_max', 'lagRet12_sic', 'ceqq', 'cheq',
       'dlttq', 'epspiq', 'saleq', 'sp500_ret_d', 'nasdaq_ret_d',
       'r2000_ret_d', 'dollar_ret_d', 'VIX', 'yield_3m', 'yield_10y',
       'gdp_growth', 'Bull_ave', 'Bull_Bear', 'ret', 'debt', 'cash', 'sale',
       'BM', 'PE', 'div_p', 'loglagPrice2', 'loglagVOL12', 'loglagMV2',
       'logatq', 'loglagVOL2']]

In [8]:
Y_test=test[['ret']]

In [9]:
#Calculate avg monthly risk free return
rf1=pd.read_excel("/Users/jimmyaspras/Downloads/Treasury bill.xlsx")
rf1['rf']=rf1['DGS3MO']/1200
rf2=rf1[['Date','rf']].dropna()
rf2['Year']=rf2['Date'].dt.year
rf2['Month']=rf2['Date'].dt.month
rf3=rf2[['Year','Month','rf']].groupby(['Year','Month'], as_index=False).mean()

In [10]:
#Import benchmark index return
indexret1=pd.read_stata("/Users/jimmyaspras/Downloads/Index return.dta")

In [11]:
#Use LassoCV to run lasso regression and use the timeseriessplit for the cross validation.  Run lasso  regression 
#using  your  new  training  sample  and  report  the  selected  value  of  Alpha  and coefficients on all 
#the selected independent variables. 

In [12]:
variable_name=X_test.columns.tolist() #get the independent variable names
tsplit=TimeSeriesSplit(n_splits=5,test_size=10000, gap=5000)
Lasso_m = LassoCV(cv=tsplit) #define the model
Lasso_m.fit(X_train,Y_train)#train the model
print("Alpha: ",Lasso_m.alpha_)

/Users/jimmyaspras/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Alpha:  87.13472491755509


In [13]:
coefficients_Lasso=pd.DataFrame(Lasso_m.coef_, columns=['coef'])
coefficients_Lasso.index=variable_name
print (coefficients_Lasso)

                            coef
sic_2              -0.000000e+00
lagRet2             0.000000e+00
lagVOL2            -8.656721e-12
lagPrice2          -0.000000e+00
lagMV2             -0.000000e+00
lagShareturnover2  -0.000000e+00
lagRet2_sic         0.000000e+00
lagRet12            0.000000e+00
lagVOL12           -8.381372e-13
lagShareturnover12 -0.000000e+00
lagRet12_std       -0.000000e+00
lagRet12_min        0.000000e+00
lagRet12_max       -0.000000e+00
lagRet12_sic        0.000000e+00
ceqq               -0.000000e+00
cheq               -0.000000e+00
dlttq              -0.000000e+00
epspiq             -0.000000e+00
saleq              -0.000000e+00
sp500_ret_d         0.000000e+00
nasdaq_ret_d        0.000000e+00
r2000_ret_d         0.000000e+00
dollar_ret_d        0.000000e+00
VIX                -0.000000e+00
yield_3m           -0.000000e+00
yield_10y          -0.000000e+00
gdp_growth         -0.000000e+00
Bull_ave            0.000000e+00
Bull_Bear           0.000000e+00
ret       

In [14]:
#Predict returns vs market
lassocoef_select=coefficients_Lasso.query("coef!=0")
Y_predictlasso=pd.DataFrame(Lasso_m.predict(X_test), columns=['Y_predict'])
Y_testlasso=pd.DataFrame(Y_test).reset_index()
lassoComb1=pd.merge(Y_testlasso, Y_predictlasso, left_index=True,right_index=True,how='inner')
lassoComb1['Year']=lassoComb1['datadate'].dt.year
lassoComb1['Month']=lassoComb1['datadate'].dt.month
lassorank1=lassoComb1[['Y_predict','Year', 'Month']].groupby(['Year','Month'],as_index=False).rank(ascending=False)
lassorank1.rename(columns={'Y_predict':'Y_predict_rank'},inplace=True)
lassostock_long1=pd.merge(lassoComb1,lassorank1,left_index=True, right_index=True)
lassostock_long2=lassostock_long1[lassostock_long1['Y_predict_rank']<=100]
lassostock_long2['datadate'].value_counts()
lassostock_long3=lassostock_long2[['ret','Year','Month']].groupby(['Year','Month']).mean()
lassostock_long4=pd.merge(lassostock_long3, rf3, left_on=['Year','Month'], right_on=['Year','Month'], how='left')
lassostock_long5=pd.merge(lassostock_long4, indexret1, left_on=['Year','Month'], right_on=['Year','Month'], how='left')
lassostock_long5['ret_rf']=lassostock_long5['ret']-lassostock_long5['rf']
lassostock_long5['ret_sp500']=lassostock_long5['ret']-lassostock_long5['sp500_ret_m']
lassostock_long5=sm.add_constant(lassostock_long5)
lassostock_long5.head()

,const,Year,Month,ret,rf,sp500_ret_m,nasdaq_ret_m,r2000_ret_m,Date,ret_rf,ret_sp500
0,1.0,2016,1,-0.017260,0.000214,-0.050735,-0.078576,-0.088486,2016-01-28,-0.017474,0.033476
1,1.0,2016,2,-0.007632,0.000262,-0.004128,-0.012137,-0.001429,2016-02-28,-0.007894,-0.003504
2,1.0,2016,3,0.022943,0.000248,0.065991,0.068430,0.077503,2016-03-28,0.022695,-0.043048
3,1.0,2016,4,0.037943,0.000190,0.002699,-0.019403,0.015098,2016-04-28,0.037753,0.035243
4,1.0,2016,5,0.017531,0.000231,0.015325,0.036163,0.021170,2016-05-28,0.017300,0.002206


In [15]:
sm.OLS(lassostock_long5[['ret']],lassostock_long5[['const']]).fit().get_robustcov_results(cov_type='HC0').summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    ret   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                       nan
Date:                Sat, 24 Jul 2021   Prob (F-statistic):                nan
Time:                        22:29:37   Log-Likelihood:                 116.33
No. Observations:                  64   AIC:                            -230.7
Df Residuals:                      63   BIC:                            -228.5
Df Model:                           0                                         
Covariance Type:                  HC0                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0106      0.005      2.149      0.036       0.001       0.020
==============================================================================
Omnibus:                       73.151   Durbin-Watson:                   1.603
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              893.864
Skew:                          -3.076   Prob(JB):                    7.94e-195
Kurtosis:                      20.244   Cond. No.                         1.00
==============================================================================

Warnings:
[1] Standard Errors are heteroscedasticity robust (HC0)
"""

In [16]:
#Return of 1.06% above the market, statistically significant with p-value of 0.036

In [17]:
#Sharpe ratio
lassoRet_rf=lassostock_long5[['ret_rf']]
lassoSR=(lassoRet_rf.mean()/lassoRet_rf.std())*np.sqrt(12)
lassoSR

ret_rf    0.841747
dtype: float64

In [18]:
#Use RidgeCV to run ridge regression and use the timeseriessplit for the cross validation. Please create your own
#candidate values for the search of Alpha. Run ridge regression using your new training  sample  and  report  
#the  selected  value  of  Alpha  and  coefficients  on  all  the  independent variables.
ridgetsplit=TimeSeriesSplit(n_splits=5,test_size=10000, gap=5000)
alpha_candidate=np.linspace(0.001,10,20)
Ridge_m = RidgeCV(alphas=alpha_candidate, cv=ridgetsplit)
Ridge_m.fit(X_train,Y_train)#train the model
Ridge_m.alpha_

/Users/jimmyaspras/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=7.69751e-24): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True,
/Users/jimmyaspras/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=7.63794e-24): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True,
/Users/jimmyaspras/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=7.72123e-24): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True,
/Users/jimmyaspras/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=7.83264e-24): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True,
/Users/jimmyaspras/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_ridge.py:

0.001

In [19]:
#Alpha 0.001

In [20]:
coefficients_Ridge=pd.DataFrame(Ridge_m.coef_).T
coefficients_Ridge.index=variable_name
print (coefficients_Ridge)

                               0
sic_2              -3.387688e-12
lagRet2             4.065479e-11
lagVOL2             4.222271e-19
lagPrice2           4.382608e-14
lagMV2              2.438675e-15
lagShareturnover2  -2.702210e-09
lagRet2_sic         1.676743e-10
lagRet12            3.604271e-08
lagVOL12           -3.491729e-19
lagShareturnover12  3.053015e-09
lagRet12_std       -2.229274e-08
lagRet12_min       -6.062082e-09
lagRet12_max        3.397773e-09
lagRet12_sic       -1.576521e-10
ceqq               -7.032788e-15
cheq                4.233701e-15
dlttq              -9.201331e-15
epspiq             -2.367641e-13
saleq              -4.395994e-15
sp500_ret_d         1.526742e-06
nasdaq_ret_d       -1.800111e-07
r2000_ret_d        -7.259178e-07
dollar_ret_d        3.271167e-07
VIX                 1.027323e-10
yield_3m            5.465529e-07
yield_10y          -9.181538e-07
gdp_growth         -1.334378e-08
Bull_ave            6.090713e-09
Bull_Bear           4.048946e-09
ret       

In [21]:
Y_predictridge=pd.DataFrame(Ridge_m.predict(X_test), columns=['Y_predict']) 
Y_testridge=pd.DataFrame(Y_test).reset_index()
ridgeComb1=pd.merge(Y_testridge, Y_predictridge, left_index=True,right_index=True,how='inner')
ridgeComb1['Year']=ridgeComb1['datadate'].dt.year
ridgeComb1['Month']=ridgeComb1['datadate'].dt.month
ridgerank1=ridgeComb1[['Y_predict','Year', 'Month']].groupby(['Year','Month'],as_index=False).rank(ascending=False)
ridgerank1.rename(columns={'Y_predict':'Y_predict_rank'},inplace=True)
ridgestock_long1=pd.merge(ridgeComb1,ridgerank1,left_index=True, right_index=True)
ridgestock_long2=ridgestock_long1[ridgestock_long1['Y_predict_rank']<=100]
ridgestock_long2['datadate'].value_counts()
ridgestock_long3=ridgestock_long2[['ret','Year','Month']].groupby(['Year','Month']).mean()
ridgestock_long4=pd.merge(ridgestock_long3, rf3, left_on=['Year','Month'], right_on=['Year','Month'], how='left')
ridgestock_long5=pd.merge(ridgestock_long4, indexret1, left_on=['Year','Month'], right_on=['Year','Month'], how='left')
ridgestock_long5['ret_rf']=ridgestock_long5['ret']-ridgestock_long5['rf']
ridgestock_long5['ret_sp500']=ridgestock_long5['ret']-ridgestock_long5['sp500_ret_m']
ridgestock_long5=sm.add_constant(ridgestock_long5)
ridgestock_long5.head()

,const,Year,Month,ret,rf,sp500_ret_m,nasdaq_ret_m,r2000_ret_m,Date,ret_rf,ret_sp500
0,1.0,2016,1,0.174353,0.000214,-0.050735,-0.078576,-0.088486,2016-01-28,0.174139,0.225088
1,1.0,2016,2,0.326144,0.000262,-0.004128,-0.012137,-0.001429,2016-02-28,0.325882,0.330273
2,1.0,2016,3,0.456612,0.000248,0.065991,0.068430,0.077503,2016-03-28,0.456364,0.390621
3,1.0,2016,4,0.372055,0.000190,0.002699,-0.019403,0.015098,2016-04-28,0.371865,0.369356
4,1.0,2016,5,0.316006,0.000231,0.015325,0.036163,0.021170,2016-05-28,0.315775,0.300681


In [22]:
sm.OLS(ridgestock_long5[['ret']],ridgestock_long5[['const']]).fit().get_robustcov_results(cov_type='HC0').summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    ret   R-squared:                      -0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                       nan
Date:                Sat, 24 Jul 2021   Prob (F-statistic):                nan
Time:                        22:30:25   Log-Likelihood:                 30.757
No. Observations:                  64   AIC:                            -59.51
Df Residuals:                      63   BIC:                            -57.36
Df Model:                           0                                         
Covariance Type:                  HC0                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.3699      0.019     19.773      0.000       0.332       0.407
==============================================================================
Omnibus:                       31.486   Durbin-Watson:                   1.344
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               62.630
Skew:                           1.668   Prob(JB):                     2.51e-14
Kurtosis:                       6.516   Cond. No.                         1.00
==============================================================================

Warnings:
[1] Standard Errors are heteroscedasticity robust (HC0)
"""

In [23]:
#The ridge regression returns a value of 36.99% average return per month with a p-value of 0

In [24]:
ridgeRet_rf=ridgestock_long5[['ret_rf']]
ridgeSR=(ridgeRet_rf.mean()/ridgeRet_rf.std())*np.sqrt(12)
ridgeSR

ret_rf    8.458599
dtype: float64

In [25]:
#Use ElasticNetCV to run elasticnet regression and use the timeseriessplit for the cross validation. Please 
#create your own candidate values for the search of l1_ratio. Run elasticnet regression using your  new  training 
#sample  and  report  the  selected  value  of  l1_ratio  and  coefficients  on  all  the selected independent 
#variables.
elasttsplit=TimeSeriesSplit(n_splits=5,test_size=10000, gap=5000)
l1_ratio_candidate=np.linspace(0.001,2,10)
Elastic_m = ElasticNetCV(l1_ratio=l1_ratio_candidate, cv=tsplit)
Elastic_m.fit(X_train,Y_train)
print("Chosen alpha value:",Elastic_m.l1_ratio_)

/Users/jimmyaspras/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Chosen alpha value: 0.001


In [26]:
coefficients_Elas=pd.DataFrame(Elastic_m.coef_,columns=['coef'])
coefficients_Elas.index=variable_name
print (coefficients_Elas)

                            coef
sic_2              -0.000000e+00
lagRet2             0.000000e+00
lagVOL2            -8.656721e-12
lagPrice2          -0.000000e+00
lagMV2             -0.000000e+00
lagShareturnover2  -0.000000e+00
lagRet2_sic         0.000000e+00
lagRet12            0.000000e+00
lagVOL12           -8.381372e-13
lagShareturnover12 -0.000000e+00
lagRet12_std       -0.000000e+00
lagRet12_min        0.000000e+00
lagRet12_max       -0.000000e+00
lagRet12_sic        0.000000e+00
ceqq               -0.000000e+00
cheq               -0.000000e+00
dlttq              -0.000000e+00
epspiq             -0.000000e+00
saleq              -0.000000e+00
sp500_ret_d         0.000000e+00
nasdaq_ret_d        0.000000e+00
r2000_ret_d         0.000000e+00
dollar_ret_d        0.000000e+00
VIX                -0.000000e+00
yield_3m           -0.000000e+00
yield_10y          -0.000000e+00
gdp_growth         -0.000000e+00
Bull_ave            0.000000e+00
Bull_Bear           0.000000e+00
ret       

In [27]:
coef_select_Elas=coefficients_Lasso.query("coef!=0")
Y_predictelast=pd.DataFrame(Elastic_m.predict(X_test), columns=['Y_predict'])
Y_testelast=pd.DataFrame(Y_test).reset_index()
elastComb1=pd.merge(Y_testelast, Y_predictelast, left_index=True,right_index=True,how='inner')
elastComb1['Year']=elastComb1['datadate'].dt.year
elastComb1['Month']=elastComb1['datadate'].dt.month
elastrank1=elastComb1[['Y_predict','Year', 'Month']].groupby(['Year','Month'],as_index=False).rank(ascending=False)
elastrank1.rename(columns={'Y_predict':'Y_predict_rank'},inplace=True)
elaststock_long1=pd.merge(elastComb1,elastrank1,left_index=True, right_index=True)
elaststock_long2=elaststock_long1[elaststock_long1['Y_predict_rank']<=100]
elaststock_long2['datadate'].value_counts()
elaststock_long3=elaststock_long2[['ret','Year','Month']].groupby(['Year','Month']).mean()
elaststock_long4=pd.merge(elaststock_long3, rf3, left_on=['Year','Month'], right_on=['Year','Month'], how='left')
elaststock_long5=pd.merge(elaststock_long4, indexret1, left_on=['Year','Month'], right_on=['Year','Month'], how='left')
elaststock_long5['ret_rf']=elaststock_long5['ret']-elaststock_long5['rf']
elaststock_long5['ret_sp500']=elaststock_long5['ret']-elaststock_long5['sp500_ret_m']
elaststock_long5=sm.add_constant(elaststock_long5)
elaststock_long5.head()

,const,Year,Month,ret,rf,sp500_ret_m,nasdaq_ret_m,r2000_ret_m,Date,ret_rf,ret_sp500
0,1.0,2016,1,-0.017260,0.000214,-0.050735,-0.078576,-0.088486,2016-01-28,-0.017474,0.033476
1,1.0,2016,2,-0.007632,0.000262,-0.004128,-0.012137,-0.001429,2016-02-28,-0.007894,-0.003504
2,1.0,2016,3,0.022943,0.000248,0.065991,0.068430,0.077503,2016-03-28,0.022695,-0.043048
3,1.0,2016,4,0.037943,0.000190,0.002699,-0.019403,0.015098,2016-04-28,0.037753,0.035243
4,1.0,2016,5,0.017531,0.000231,0.015325,0.036163,0.021170,2016-05-28,0.017300,0.002206


In [28]:
sm.OLS(elaststock_long5[['ret']],elaststock_long5[['const']]).fit().get_robustcov_results(cov_type='HC0').summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    ret   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                       nan
Date:                Sat, 24 Jul 2021   Prob (F-statistic):                nan
Time:                        22:31:03   Log-Likelihood:                 116.33
No. Observations:                  64   AIC:                            -230.7
Df Residuals:                      63   BIC:                            -228.5
Df Model:                           0                                         
Covariance Type:                  HC0                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0106      0.005      2.149      0.036       0.001       0.020
==============================================================================
Omnibus:                       73.151   Durbin-Watson:                   1.603
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              893.864
Skew:                          -3.076   Prob(JB):                    7.94e-195
Kurtosis:                      20.244   Cond. No.                         1.00
==============================================================================

Warnings:
[1] Standard Errors are heteroscedasticity robust (HC0)
"""

In [29]:
elastRet_rf=elaststock_long5[['ret_rf']]
elastSR=(elastRet_rf.mean()/elastRet_rf.std())*np.sqrt(12)
elastSR

ret_rf    0.841747
dtype: float64

In [30]:
#Results of ElasticNetCV are identical to lasso regression with a monthly return of 1.06 and, p-value of 0.036
#and sharpe ratio of 0.841747